In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import tqdm
import operator
import os
import os.path
from tqdm import tqdm_notebook as tqdm
from imgpy import Img
import pickle
from skimage import io
import csv
import re
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
import scipy
from scipy import misc
from keras import regularizers
from keras.layers import Input
from keras.layers import LSTM, RepeatVector
from skimage.transform import resize
from skimage import io
from sklearn.mixture import GaussianMixture
import glob
import math
import keras

from skimage.viewer import ImageViewer
from keras.callbacks import LearningRateScheduler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words
from keras.preprocessing import image
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import utils
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:
#Define Paths
image_source_path = '/Users/kmotwani/Dropbox/AM207 Paper/download_thumbs/'
text_source_path = '/Users/kmotwani/Dropbox/Data Science 209b Final Project/TGIF-Release/data/tgif-v1.0-filtered.tsv'

In [3]:
#Helper Function to get images from path
def get_df(text_source_path, image_source_path, thresh):
    count = 0
    with open(text_source_path) as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        final = []
        for row in rd:
            temp_dict, file = {}, row[0].split('/')[-1][:-4]
            temp_dict['File'] = file
            temp_dict['Caption'] = row[1]
            try:
                temp_img = image.load_img(image_source_path+file+'.png')
            except FileNotFoundError:
                continue
            temp_dict['Image'] = np.array(temp_img)
            final.append(temp_dict)
            count +=1 
            if count==thresh:
                break
    return pd.DataFrame(final)


#Get Images from User List and Path
df = get_df(text_source_path, image_source_path, 100000)
print("Number of images loaded:", len(df))
display(df.head())

Number of images loaded: 100000


,Caption,File,Image
0,"a man is glaring, and someone with sunglasses ...",tumblr_mevmyaKtDf1rgvhr8o1_500,"[[[169, 164, 149], [169, 163, 148], [169, 165,..."
1,a cat tries to catch a mouse on a tablet,tumblr_nok4eeONTv1s2yegdo1_400,"[[[170, 157, 124], [164, 150, 123], [160, 145,..."
2,a man dressed in red is dancing.,tumblr_mllh01J96X1s9npefo1_250,"[[[102, 88, 72], [103, 88, 67], [102, 88, 71],..."
3,an animal comes close to another in the jungle,tumblr_nqlr0rn8ox1r2r0koo1_400,"[[[98, 102, 102], [105, 108, 107], [94, 98, 99..."
4,a man in a hat adjusts his tie and makes a wei...,tumblr_mvtuwlhSkE1qbnleeo1_500,"[[[73, 73, 73], [74, 74, 74], [74, 74, 74], [6..."


In [4]:
#Filter dataset to images with "a man is" in the caption
df_filtered = df[df['Caption'].str.contains("a man is")]
display(df_filtered)
print("Length od filtered dataframe:",len(df_filtered))

,Caption,File,Image
0,"a man is glaring, and someone with sunglasses ...",tumblr_mevmyaKtDf1rgvhr8o1_500,"[[[169, 164, 149], [169, 163, 148], [169, 165,..."
36,a man is wearing a cap and headphones making h...,tumblr_nnxshbEKOc1rqd7uco1_500,"[[[198, 197, 198], [172, 171, 172], [144, 143,..."
49,"a man is sitting in a room, he has a gun on hi...",tumblr_nqir72vsUx1sd7dido1_500,"[[[64, 64, 64], [72, 72, 72], [43, 43, 43], [3..."
70,a man is walking in a crowd in the dark.,tumblr_nqgz75OOEB1snj1pfo1_500,"[[[72, 43, 20], [72, 44, 21], [72, 46, 23], [7..."
78,a man is sitting on a sofa beside three others...,tumblr_nkymseZ2UC1u4gszco1_400,"[[[163, 181, 181], [168, 186, 191], [163, 182,..."
100,"a man is walking on the rain, wearing a hat",tumblr_nppe4itAAr1sz4xdoo1_500,"[[[32, 32, 32], [34, 34, 34], [33, 33, 33], [3..."
105,a man is speaking with someone and smiling.,tumblr_mpehmq9Xmk1s6our6o1_500,"[[[60, 9, 26], [68, 13, 26], [59, 15, 38], [73..."
154,"a man is laughing, he puts his head on his arm",tumblr_nqphwc3BZW1uzvps5o1_250,"[[[87, 87, 87], [83, 83, 83], [86, 86, 86], [7..."
182,"a man is crying, he has a ring on his ring finger",tumblr_npcto9llWY1un7teyo1_500,"[[[48, 48, 48], [72, 72, 72], [79, 79, 79], [8..."
199,a man is dancing over a gangway.,tumblr_nodrisQbvZ1rs4x9yo1_400,"[[[22, 98, 142], [25, 104, 148], [75, 108, 144..."


Length od filtered dataframe: 9846


In [5]:
save_image_path = '/Users/kmotwani/Desktop/Me/Education/Courses/APCOMP 209B/Dataset/'
save_text_path = '/Users/kmotwani/Desktop/Me/Education/Courses/APCOMP 209B/Dataset/'

In [8]:
import ObjectDetection
model_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/keras-retinanet/snapshots/resnet50_coco_best.h5"
model = ObjectDetection.ImageDetect(model_path)

/anaconda3/lib/python3.6/site-packages/keras/models.py:274: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.
  sample_weight_mode=sample_weight_mode)


In [12]:
#Save Filtered Dataframe
for i in range(len(df_filtered)):
    print(i+1,"/",len(df_filtered))
    temp_count = model.predict(df_filtered.iloc[i,:]['Image'])['person']
    if temp_count!=1:
        continue
    temp_image, temp_caption, temp_file = df_filtered.iloc[i,:]['Image'], df_filtered.iloc[i,:]['Caption'], df_filtered.iloc[i,:]['File']
    temp_path = save_text_path+temp_file+'.txt'
    new = open(temp_path, 'w')
    count = 0
    for word in temp_caption.split():
        if os.path.exists(temp_path):
            new.write(word+" ")
            count += 1
            if count==7:
                break
    new.close()
    temp_path = save_image_path+temp_file+'.jpg'
    io.imsave(temp_path, temp_image)

1 / 9846
Processing Time :  4.2395288944244385
2 / 9846
Processing Time :  4.441968202590942
3 / 9846
Processing Time :  4.765834093093872
4 / 9846
Processing Time :  4.678600072860718
5 / 9846
Processing Time :  4.736582040786743
6 / 9846
Processing Time :  4.7136921882629395
7 / 9846
Processing Time :  4.749958038330078
8 / 9846
Processing Time :  4.363470077514648
9 / 9846
Processing Time :  4.250452995300293
10 / 9846
Processing Time :  4.510741949081421
11 / 9846
Processing Time :  4.743856906890869
12 / 9846
Processing Time :  4.666827917098999
13 / 9846
Processing Time :  4.848880290985107
14 / 9846
Processing Time :  5.283918142318726
15 / 9846
Processing Time :  6.063146114349365
16 / 9846
Processing Time :  5.047680854797363
17 / 9846
Processing Time :  6.007863998413086
18 / 9846
Processing Time :  5.301580905914307
19 / 9846
Processing Time :  4.734943866729736
20 / 9846
Processing Time :  5.007622003555298
21 / 9846
Processing Time :  4.526875972747803
22 / 9846
Processing

174 / 9846
Processing Time :  4.39369010925293
175 / 9846
Processing Time :  4.424823045730591
176 / 9846
Processing Time :  4.848176956176758
177 / 9846
Processing Time :  4.2572009563446045
178 / 9846
Processing Time :  4.246136903762817
179 / 9846
Processing Time :  4.243654727935791
180 / 9846
Processing Time :  4.317368984222412
181 / 9846
Processing Time :  4.243713140487671
182 / 9846
Processing Time :  4.292701959609985
183 / 9846
Processing Time :  4.313593864440918
184 / 9846
Processing Time :  4.272610187530518
185 / 9846
Processing Time :  4.280200004577637
186 / 9846
Processing Time :  4.28139591217041
187 / 9846
Processing Time :  4.286418914794922
188 / 9846
Processing Time :  4.256518840789795
189 / 9846
Processing Time :  4.2915661334991455
190 / 9846
Processing Time :  4.3142170906066895
191 / 9846
Processing Time :  4.292036056518555
192 / 9846
Processing Time :  4.286364316940308
193 / 9846
Processing Time :  4.275133848190308
194 / 9846
Processing Time :  4.2672920

345 / 9846
Processing Time :  4.299973249435425
346 / 9846
Processing Time :  4.300769090652466
347 / 9846
Processing Time :  4.489046096801758
348 / 9846
Processing Time :  4.2723188400268555
349 / 9846
Processing Time :  4.318774938583374
350 / 9846
Processing Time :  4.389392137527466
351 / 9846
Processing Time :  4.406776189804077
352 / 9846
Processing Time :  4.5335798263549805
353 / 9846
Processing Time :  4.360558986663818
354 / 9846
Processing Time :  4.31506609916687
355 / 9846
Processing Time :  4.336221933364868
356 / 9846
Processing Time :  4.299299001693726
357 / 9846
Processing Time :  4.308387041091919
358 / 9846
Processing Time :  4.33766508102417
359 / 9846
Processing Time :  4.987910985946655
360 / 9846
Processing Time :  4.7278831005096436
361 / 9846
Processing Time :  4.280319929122925
362 / 9846
Processing Time :  4.330377817153931
363 / 9846
Processing Time :  4.372726917266846
364 / 9846
Processing Time :  4.3090009689331055
365 / 9846
Processing Time :  4.290220

Processing Time :  4.425094127655029
517 / 9846
Processing Time :  4.702821969985962
518 / 9846
Processing Time :  4.822505950927734
519 / 9846
Processing Time :  5.006670951843262
520 / 9846
Processing Time :  4.397280693054199
521 / 9846
Processing Time :  4.374994993209839
522 / 9846
Processing Time :  4.317266941070557
523 / 9846
Processing Time :  4.35132908821106
524 / 9846
Processing Time :  4.430717945098877
525 / 9846
Processing Time :  4.4116761684417725
526 / 9846
Processing Time :  4.494351863861084
527 / 9846
Processing Time :  4.698569297790527
528 / 9846
Processing Time :  4.629560947418213
529 / 9846
Processing Time :  4.499628067016602
530 / 9846
Processing Time :  4.319907188415527
531 / 9846
Processing Time :  4.3438098430633545
532 / 9846
Processing Time :  4.330069065093994
533 / 9846
Processing Time :  4.589354038238525
534 / 9846
Processing Time :  5.01241397857666
535 / 9846
Processing Time :  4.833960056304932
536 / 9846
Processing Time :  4.695134878158569
537

687 / 9846
Processing Time :  6.05173921585083
688 / 9846
Processing Time :  5.304579973220825
689 / 9846
Processing Time :  5.4328320026397705
690 / 9846
Processing Time :  5.071892023086548
691 / 9846
Processing Time :  5.936734914779663
692 / 9846
Processing Time :  5.482372045516968
693 / 9846
Processing Time :  5.015419960021973
694 / 9846
Processing Time :  5.038463354110718
695 / 9846
Processing Time :  5.012126207351685
696 / 9846
Processing Time :  5.1354639530181885
697 / 9846
Processing Time :  5.006306171417236
698 / 9846
Processing Time :  5.002530097961426
699 / 9846
Processing Time :  5.0311408042907715
700 / 9846
Processing Time :  5.107563018798828
701 / 9846
Processing Time :  5.015766859054565
702 / 9846
Processing Time :  5.030629873275757
703 / 9846
Processing Time :  4.987642288208008
704 / 9846
Processing Time :  4.983231782913208
705 / 9846
Processing Time :  5.01153826713562
706 / 9846
Processing Time :  5.076460123062134
707 / 9846
Processing Time :  5.2658631

...

Processing Time :  5.258500814437866
6889 / 9846
Processing Time :  5.143239974975586
6890 / 9846
Processing Time :  4.683261871337891
6891 / 9846
Processing Time :  4.782510995864868
6892 / 9846
Processing Time :  4.798811912536621
6893 / 9846
Processing Time :  4.732997179031372
6894 / 9846
Processing Time :  4.6200151443481445
6895 / 9846
Processing Time :  4.870962858200073
6896 / 9846
Processing Time :  4.616691827774048
6897 / 9846
Processing Time :  4.7792067527771
6898 / 9846
Processing Time :  4.778766870498657
6899 / 9846
Processing Time :  4.788925886154175
6900 / 9846
Processing Time :  4.709661960601807
6901 / 9846
Processing Time :  4.886089086532593
6902 / 9846
Processing Time :  4.631052017211914
6903 / 9846
Processing Time :  4.670212984085083
6904 / 9846
Processing Time :  4.675793886184692
6905 / 9846
Processing Time :  4.808679103851318
6906 / 9846
Processing Time :  4.6781699657440186
6907 / 9846
Processing Time :  4.724097728729248
6908 / 9846
Processing Time :  4

7056 / 9846
Processing Time :  4.776327848434448
7057 / 9846
Processing Time :  4.654092788696289
7058 / 9846
Processing Time :  4.826984882354736
7059 / 9846
Processing Time :  4.628874063491821
7060 / 9846
Processing Time :  4.888513803482056
7061 / 9846
Processing Time :  4.83322811126709
7062 / 9846
Processing Time :  4.849472999572754
7063 / 9846
Processing Time :  4.875274181365967
7064 / 9846
Processing Time :  4.6459338665008545
7065 / 9846
Processing Time :  4.674901962280273
7066 / 9846
Processing Time :  4.9128501415252686
7067 / 9846
Processing Time :  4.67478609085083
7068 / 9846
Processing Time :  4.799889087677002
7069 / 9846
Processing Time :  4.882759094238281
7070 / 9846
Processing Time :  4.807681083679199
7071 / 9846
Processing Time :  4.608207941055298
7072 / 9846
Processing Time :  4.896918058395386
7073 / 9846
Processing Time :  4.634747266769409
7074 / 9846
Processing Time :  4.651517868041992
7075 / 9846
Processing Time :  4.824876070022583
7076 / 9846
Processi

Processing Time :  5.376161098480225
7224 / 9846
Processing Time :  4.752701997756958
7225 / 9846
Processing Time :  4.790429353713989
7226 / 9846
Processing Time :  4.590342283248901
7227 / 9846
Processing Time :  4.618531942367554
7228 / 9846
Processing Time :  4.5799291133880615
7229 / 9846
Processing Time :  4.78451132774353
7230 / 9846
Processing Time :  5.215224027633667
7231 / 9846
Processing Time :  5.187502145767212
7232 / 9846
Processing Time :  4.624479055404663
7233 / 9846
Processing Time :  4.822613954544067
7234 / 9846
Processing Time :  4.673403978347778
7235 / 9846
Processing Time :  4.8692710399627686
7236 / 9846
Processing Time :  4.850987911224365
7237 / 9846
Processing Time :  4.80862021446228
7238 / 9846
Processing Time :  4.802491903305054
7239 / 9846
Processing Time :  4.734632253646851
7240 / 9846
Processing Time :  4.612652063369751
7241 / 9846
Processing Time :  4.847342014312744
7242 / 9846
Processing Time :  4.702849864959717
7243 / 9846
Processing Time :  5

Processing Time :  4.154988050460815
7391 / 9846
Processing Time :  4.163900852203369
7392 / 9846
Processing Time :  4.15527606010437
7393 / 9846
Processing Time :  4.103994846343994
7394 / 9846
Processing Time :  4.122072219848633
7395 / 9846
Processing Time :  4.068143844604492
7396 / 9846
Processing Time :  3.966233253479004
7397 / 9846
Processing Time :  4.164369106292725
7398 / 9846
Processing Time :  4.211055755615234
7399 / 9846
Processing Time :  4.137635231018066
7400 / 9846
Processing Time :  4.131294012069702
7401 / 9846
Processing Time :  4.162768840789795
7402 / 9846
Processing Time :  3.9604930877685547
7403 / 9846
Processing Time :  3.972787857055664
7404 / 9846
Processing Time :  3.894078016281128
7405 / 9846
Processing Time :  4.019995927810669
7406 / 9846
Processing Time :  4.10591197013855
7407 / 9846
Processing Time :  4.087066173553467
7408 / 9846
Processing Time :  3.9390580654144287
7409 / 9846
Processing Time :  4.115992069244385
7410 / 9846
Processing Time :  3

KeyboardInterrupt: 